In [1]:
import gradio as gr
import cv2
import numpy as np
import json
import os
from PIL import Image, ImageDraw
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.backends.backend_agg import FigureCanvasAgg

In [2]:
class AnnotationReviewer:
    def __init__(self, annotation_file_path, image_base_path):
        self.annotation_file = annotation_file_path
        self.image_base_path = Path(image_base_path)
        self.annotations = {}
        self.current_image = None
        self.current_image_name = None
        self.modified = False
        self.reviewed_images = set()  # Track reviewed images
        self.modifications_history = {}  # Track modifications per image
        
        # Load annotations
        self.load_annotations()
        
    def load_annotations(self):
        """Load annotations from JSON file"""
        if os.path.exists(self.annotation_file):
            with open(self.annotation_file, 'r', encoding='utf-8') as f:
                self.annotations = json.load(f)
                
            # Load reviewed status if exists
            for img_name, data in self.annotations.items():
                if data.get('reviewed', False):
                    self.reviewed_images.add(img_name)
        else:
            print(f"Annotation file not found: {self.annotation_file}")
            
    def save_annotations(self):
        """Save annotations back to JSON file"""
        # Mark reviewed images in annotations
        for img_name in self.reviewed_images:
            if img_name in self.annotations:
                self.annotations[img_name]['reviewed'] = True
                
        with open(self.annotation_file, 'w', encoding='utf-8') as f:
            json.dump(self.annotations, f, indent=2, ensure_ascii=False)
        self.modified = False
        print("Annotations saved successfully!")
        
    def mark_as_reviewed(self, image_name):
        """Mark image as reviewed"""
        self.reviewed_images.add(image_name)
        if image_name not in self.modifications_history:
            self.modifications_history[image_name] = []
        self.modifications_history[image_name].append("Marked as reviewed")
        
    def get_image_list(self):
        """Get list of images with annotations"""
        return list(self.annotations.keys())
        
    def get_review_status(self):
        """Get review status summary"""
        total = len(self.annotations)
        reviewed = len(self.reviewed_images)
        modified = len(self.modifications_history)
        return f"Total: {total} | Reviewed: {reviewed} | Modified: {modified}"
        
    def get_image_status(self, image_name):
        """Get status for a specific image"""
        status = []
        if image_name in self.reviewed_images:
            status.append("✅ Reviewed")
        if image_name in self.modifications_history:
            status.append(f"🔄 Modified ({len(self.modifications_history[image_name])} changes)")
        return " | ".join(status) if status else "❌ Not reviewed"
        
    def load_image_with_annotations(self, image_name):
        """Load image and draw current annotations"""
        if image_name not in self.annotations:
            return None, "Image not found in annotations"
            
        # Find image file
        image_path = None
        for ext in ['.jpg', '.png', '.PNG', '.JPG']:
            potential_path = self.image_base_path / image_name.replace('.png', ext).replace('.jpg', ext)
            if potential_path.exists():
                image_path = potential_path
                break
                
        if image_path is None:
            # Try recursive search
            for img_file in self.image_base_path.rglob(image_name):
                if img_file.exists():
                    image_path = img_file
                    break
                    
        if image_path is None:
            return None, f"Image file not found: {image_name}"
            
        # Load image
        img = cv2.imread(str(image_path))
        if img is None:
            return None, f"Cannot load image: {image_path}"
            
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.current_image = img_rgb.copy()
        self.current_image_name = image_name
        
        # Draw annotations
        annotated_img = self.draw_annotations(img_rgb, image_name)
        
        # Get annotation info
        ann_data = self.annotations[image_name]
        sig_count = len(ann_data.get('signatures', []))
        non_sig_count = len(ann_data.get('non_signatures', []))
        
        info = f"Signatures: {sig_count}, Non-signatures: {non_sig_count}"
        
        return annotated_img, info
        
    def draw_annotations(self, img, image_name):
        """Draw bounding boxes on image"""
        img_draw = img.copy()
        ann_data = self.annotations.get(image_name, {})
        
        # Draw signatures (green)
        for i, sig in enumerate(ann_data.get('signatures', [])):
            x, y, w, h = sig['bbox']
            cv2.rectangle(img_draw, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(img_draw, f"SIG-{i}", (x, y-5), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
            
        # Draw non-signatures (red)
        for i, non_sig in enumerate(ann_data.get('non_signatures', [])):
            x, y, w, h = non_sig['bbox']
            cv2.rectangle(img_draw, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(img_draw, f"NO-{i}", (x, y-5), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
            
        return img_draw
        
    def delete_annotation(self, image_name, annotation_type, index):
        """Delete a specific annotation"""
        if image_name not in self.annotations:
            return "Image not found"
            
        ann_data = self.annotations[image_name]
        
        if annotation_type == "signature" and index < len(ann_data.get('signatures', [])):
            del ann_data['signatures'][index]
            self.modified = True
            self._track_modification(image_name, f"Deleted signature {index}")
            return f"Deleted signature {index}"
        elif annotation_type == "non_signature" and index < len(ann_data.get('non_signatures', [])):
            del ann_data['non_signatures'][index]
            self.modified = True
            self._track_modification(image_name, f"Deleted non-signature {index}")
            return f"Deleted non-signature {index}"
        else:
            return "Invalid annotation index"
            
    def move_annotation(self, image_name, from_type, from_index, to_type):
        """Move annotation between signature and non-signature"""
        if image_name not in self.annotations:
            return "Image not found"
            
        ann_data = self.annotations[image_name]
        
        # Get the annotation
        if from_type == "signature" and from_index < len(ann_data.get('signatures', [])):
            annotation = ann_data['signatures'].pop(from_index)
        elif from_type == "non_signature" and from_index < len(ann_data.get('non_signatures', [])):
            annotation = ann_data['non_signatures'].pop(from_index)
        else:
            return "Invalid source annotation"
            
        # Add to destination
        if to_type == "signature":
            if 'signatures' not in ann_data:
                ann_data['signatures'] = []
            ann_data['signatures'].append(annotation)
        else:
            if 'non_signatures' not in ann_data:
                ann_data['non_signatures'] = []
            ann_data['non_signatures'].append(annotation)
            
        self.modified = True
        self._track_modification(image_name, f"Moved {from_type} {from_index} to {to_type}")
        return f"Moved {from_type} {from_index} to {to_type}"
        
    def edit_bbox(self, image_name, annotation_type, index, new_x, new_y, new_w, new_h):
        """Edit bounding box coordinates"""
        if image_name not in self.annotations:
            return "Image not found"
            
        ann_data = self.annotations[image_name]
        
        if annotation_type == "signature" and index < len(ann_data.get('signatures', [])):
            old_bbox = ann_data['signatures'][index]['bbox']
            ann_data['signatures'][index]['bbox'] = [int(new_x), int(new_y), int(new_w), int(new_h)]
            self.modified = True
            self._track_modification(image_name, f"Edited signature {index} bbox from {old_bbox} to [{new_x}, {new_y}, {new_w}, {new_h}]")
            return f"Updated signature {index} bbox"
        elif annotation_type == "non_signature" and index < len(ann_data.get('non_signatures', [])):
            old_bbox = ann_data['non_signatures'][index]['bbox']
            ann_data['non_signatures'][index]['bbox'] = [int(new_x), int(new_y), int(new_w), int(new_h)]
            self.modified = True
            self._track_modification(image_name, f"Edited non-signature {index} bbox from {old_bbox} to [{new_x}, {new_y}, {new_w}, {new_h}]")
            return f"Updated non-signature {index} bbox"
        else:
            return "Invalid annotation index"
            
    def _track_modification(self, image_name, modification):
        """Track modifications for an image"""
        if image_name not in self.modifications_history:
            self.modifications_history[image_name] = []
        self.modifications_history[image_name].append(modification)
        
    def get_modification_history(self, image_name):
        """Get modification history for an image"""
        if image_name in self.modifications_history:
            return "\n".join(self.modifications_history[image_name])
        return "No modifications"

In [3]:
# Initialize the reviewer (update paths as needed)
reviewer = AnnotationReviewer(
    annotation_file_path="auto_labels/auto_annotations.json",
    image_base_path="."  # Base path to search for images
)

def load_image_interface(image_name):
    """Load and display image with annotations"""
    if not image_name:
        return None, "Please select an image", reviewer.get_review_status(), ""
    
    img, info = reviewer.load_image_with_annotations(image_name)
    status = reviewer.get_image_status(image_name)
    review_summary = reviewer.get_review_status()
    return img, info, review_summary, status

def delete_annotation_interface(image_name, ann_type, ann_index):
    """Delete annotation interface"""
    if not image_name:
        return None, "No image selected", reviewer.get_review_status(), ""
    
    try:
        ann_index = int(ann_index)
        message = reviewer.delete_annotation(image_name, ann_type, ann_index)
        
        # Reload image
        img, info = reviewer.load_image_with_annotations(image_name)
        status = reviewer.get_image_status(image_name)
        review_summary = reviewer.get_review_status()
        return img, f"{info} | {message}", review_summary, status
    except ValueError:
        return None, "Invalid index number", reviewer.get_review_status(), ""

def move_annotation_interface(image_name, from_type, from_index, to_type):
    """Move annotation between categories"""
    if not image_name:
        return None, "No image selected", reviewer.get_review_status(), ""
    
    try:
        from_index = int(from_index)
        message = reviewer.move_annotation(image_name, from_type, from_index, to_type)
        
        # Reload image
        img, info = reviewer.load_image_with_annotations(image_name)
        status = reviewer.get_image_status(image_name)
        review_summary = reviewer.get_review_status()
        return img, f"{info} | {message}", review_summary, status
    except ValueError:
        return None, "Invalid index number", reviewer.get_review_status(), ""

def edit_bbox_interface(image_name, ann_type, ann_index, new_x, new_y, new_w, new_h):
    """Edit bounding box interface"""
    if not image_name:
        return None, "No image selected", reviewer.get_review_status(), ""
    
    try:
        ann_index = int(ann_index)
        message = reviewer.edit_bbox(image_name, ann_type, ann_index, new_x, new_y, new_w, new_h)
        
        # Reload image
        img, info = reviewer.load_image_with_annotations(image_name)
        status = reviewer.get_image_status(image_name)
        review_summary = reviewer.get_review_status()
        return img, f"{info} | {message}", review_summary, status
    except ValueError:
        return None, "Invalid input values", reviewer.get_review_status(), ""

def mark_reviewed_interface(image_name):
    """Mark image as reviewed"""
    if not image_name:
        return reviewer.get_review_status(), ""
    
    reviewer.mark_as_reviewed(image_name)
    status = reviewer.get_image_status(image_name)
    review_summary = reviewer.get_review_status()
    return review_summary, status

def save_annotations_interface():
    """Save annotations"""
    reviewer.save_annotations()
    return "Annotations saved successfully!"

def get_annotation_details(image_name):
    """Get detailed annotation information"""
    if not image_name or image_name not in reviewer.annotations:
        return "No annotations found"
    
    ann_data = reviewer.annotations[image_name]
    details = []
    
    # Signatures
    for i, sig in enumerate(ann_data.get('signatures', [])):
        x, y, w, h = sig['bbox']
        conf = sig.get('confidence', 'N/A')
        details.append(f"SIG-{i}: ({x}, {y}, {w}, {h}) conf={conf}")
    
    # Non-signatures  
    for i, non_sig in enumerate(ann_data.get('non_signatures', [])):
        x, y, w, h = non_sig['bbox']
        conf = non_sig.get('confidence', 'N/A')
        details.append(f"NO-{i}: ({x}, {y}, {w}, {h}) conf={conf}")
    
    return "\n".join(details) if details else "No annotations"

def get_modification_history_interface(image_name):
    """Get modification history for selected image"""
    if not image_name:
        return "No image selected"
    return reviewer.get_modification_history(image_name)

In [4]:
# Create Gradio Interface
with gr.Blocks(title="Signature Annotation Reviewer") as interface:
    gr.Markdown("# 📝 Signature Annotation Reviewer")
    gr.Markdown("Review and edit signature detection annotations")
    
    # Review status summary
    with gr.Row():
        review_summary = gr.Textbox(
            label="📊 Review Summary", 
            value=reviewer.get_review_status(),
            interactive=False
        )
    
    with gr.Row():
        with gr.Column(scale=1):
            # Image selection
            image_dropdown = gr.Dropdown(
                choices=reviewer.get_image_list(),
                label="Select Image",
                value=reviewer.get_image_list()[0] if reviewer.get_image_list() else None
            )
            
            # Load image button
            load_btn = gr.Button("📂 Load Image", variant="primary")
            
            # Image status
            image_status = gr.Textbox(label="📋 Image Status", interactive=False)
            
            # Mark as reviewed
            mark_reviewed_btn = gr.Button("✅ Mark as Reviewed", variant="secondary")
            
            # Image info
            info_text = gr.Textbox(label="Image Info", interactive=False)
            
            # Annotation details
            details_text = gr.Textbox(
                label="Annotation Details", 
                lines=5, 
                interactive=False,
                placeholder="Select an image to see annotation details"
            )
            
            # Modification history
            history_text = gr.Textbox(
                label="📝 Modification History",
                lines=4,
                interactive=False,
                placeholder="Modification history will appear here"
            )
            
        with gr.Column(scale=2):
            # Image display
            image_display = gr.Image(label="Image with Annotations", type="numpy")
    
    gr.Markdown("## 🛠️ Edit Annotations")
    
    with gr.Row():
        with gr.Column():
            gr.Markdown("### ❌ Delete Annotation")
            with gr.Row():
                delete_type = gr.Radio(
                    choices=["signature", "non_signature"], 
                    label="Type",
                    value="signature"
                )
                delete_index = gr.Number(
                    label="Index", 
                    value=0, 
                    precision=0,
                    minimum=0
                )
            delete_btn = gr.Button("🗑️ Delete", variant="secondary")
            
        with gr.Column():
            gr.Markdown("### 🔄 Move Annotation")
            with gr.Row():
                move_from_type = gr.Radio(
                    choices=["signature", "non_signature"], 
                    label="From Type",
                    value="signature"
                )
                move_from_index = gr.Number(
                    label="From Index", 
                    value=0, 
                    precision=0,
                    minimum=0
                )
            move_to_type = gr.Radio(
                choices=["signature", "non_signature"], 
                label="To Type",
                value="non_signature"
            )
            move_btn = gr.Button("↔️ Move", variant="secondary")
    
    gr.Markdown("### ✏️ Edit Bounding Box")
    with gr.Row():
        with gr.Column():
            edit_type = gr.Radio(
                choices=["signature", "non_signature"], 
                label="Annotation Type",
                value="signature"
            )
            edit_index = gr.Number(
                label="Index", 
                value=0, 
                precision=0,
                minimum=0
            )
        with gr.Column():
            with gr.Row():
                edit_x = gr.Number(label="X", value=0, precision=0)
                edit_y = gr.Number(label="Y", value=0, precision=0)
            with gr.Row():
                edit_w = gr.Number(label="Width", value=50, precision=0, minimum=1)
                edit_h = gr.Number(label="Height", value=50, precision=0, minimum=1)
        edit_bbox_btn = gr.Button("📐 Update BBox", variant="secondary")
    
    with gr.Row():
        save_btn = gr.Button("💾 Save All Changes", variant="primary", size="lg")
        save_status = gr.Textbox(label="Save Status", interactive=False)
    
    # Event handlers
    def update_details_and_history(image_name):
        details = get_annotation_details(image_name)
        history = get_modification_history_interface(image_name)
        return details, history
    
    def load_and_update_all(image_name):
        img, info, summary, status = load_image_interface(image_name)
        details, history = update_details_and_history(image_name)
        return img, info, summary, status, details, history
    
    load_btn.click(
        fn=load_and_update_all,
        inputs=[image_dropdown],
        outputs=[image_display, info_text, review_summary, image_status, details_text, history_text]
    )
    
    image_dropdown.change(
        fn=load_and_update_all,
        inputs=[image_dropdown],
        outputs=[image_display, info_text, review_summary, image_status, details_text, history_text]
    )
    
    mark_reviewed_btn.click(
        fn=mark_reviewed_interface,
        inputs=[image_dropdown],
        outputs=[review_summary, image_status]
    ).then(
        fn=get_modification_history_interface,
        inputs=[image_dropdown],
        outputs=[history_text]
    )
    
    delete_btn.click(
        fn=delete_annotation_interface,
        inputs=[image_dropdown, delete_type, delete_index],
        outputs=[image_display, info_text, review_summary, image_status]
    ).then(
        fn=update_details_and_history,
        inputs=[image_dropdown],
        outputs=[details_text, history_text]
    )
    
    move_btn.click(
        fn=move_annotation_interface,
        inputs=[image_dropdown, move_from_type, move_from_index, move_to_type],
        outputs=[image_display, info_text, review_summary, image_status]
    ).then(
        fn=update_details_and_history,
        inputs=[image_dropdown],
        outputs=[details_text, history_text]
    )
    
    edit_bbox_btn.click(
        fn=edit_bbox_interface,
        inputs=[image_dropdown, edit_type, edit_index, edit_x, edit_y, edit_w, edit_h],
        outputs=[image_display, info_text, review_summary, image_status]
    ).then(
        fn=update_details_and_history,
        inputs=[image_dropdown],
        outputs=[details_text, history_text]
    )
    
    save_btn.click(
        fn=save_annotations_interface,
        inputs=[],
        outputs=[save_status]
    )

# Launch the interface
if __name__ == "__main__":
    interface.launch(share=True, debug=True)

* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Keyboard interruption in main thread... closing server.


In [5]:
interface.close()

Closing server running on port: 7860


# 📋 Hướng dẫn sử dụng Annotation Reviewer (Nâng cao)

## Tính năng chính:

### 🔍 **Xem và quản lý annotations**
1. **Xem ảnh với annotations**: Chọn ảnh từ dropdown để xem các vùng được khoanh
2. **Theo dõi trạng thái duyệt**: Hiển thị tổng quan và trạng thái từng ảnh
3. **Lịch sử thay đổi**: Theo dõi tất cả các chỉnh sửa đã thực hiện

### ✏️ **Chỉnh sửa annotations**
1. **Xóa annotation sai**: Chọn loại và chỉ số để xóa
2. **Chuyển đổi annotation**: Di chuyển giữa signature và non-signature
3. **📐 Chỉnh sửa bounding box**: Thay đổi tọa độ và kích thước vùng khoanh
4. **✅ Đánh dấu đã duyệt**: Đánh dấu ảnh đã được kiểm tra

### 💾 **Lưu trữ và theo dõi**
1. **Lưu thay đổi**: Lưu tất cả chỉnh sửa vào file JSON
2. **Trạng thái tổng quan**: Hiển thị số ảnh đã duyệt/chỉnh sửa
3. **Lịch sử chi tiết**: Xem tất cả thay đổi cho từng ảnh

## Cách sử dụng:

### 🚀 **Khởi động**
1. Chạy cell đầu tiên để cài đặt packages
2. Chạy các cell tiếp theo để khởi tạo
3. Cập nhật đường dẫn file annotation và thư mục ảnh nếu cần

### 🎯 **Duyệt annotations**
1. **Chọn ảnh**: Dùng dropdown để chọn ảnh
2. **Xem thông tin**: 
   - 📊 **Review Summary**: Tổng quan tiến độ duyệt
   - 📋 **Image Status**: Trạng thái ảnh hiện tại
   - **Annotation Details**: Chi tiết các vùng khoanh
   - 📝 **Modification History**: Lịch sử thay đổi

### 🛠️ **Chỉnh sửa**

#### ❌ **Xóa annotation**
1. Chọn Type (signature/non_signature)
2. Nhập Index (bắt đầu từ 0)
3. Nhấn "🗑️ Delete"

#### 🔄 **Di chuyển annotation**
1. Chọn From Type và From Index
2. Chọn To Type
3. Nhấn "↔️ Move"

#### 📐 **Chỉnh sửa bounding box**
1. Chọn Annotation Type và Index
2. Nhập tọa độ mới:
   - **X, Y**: Tọa độ góc trái trên
   - **Width, Height**: Chiều rộng và cao
3. Nhấn "📐 Update BBox"

#### ✅ **Đánh dấu hoàn thành**
- Nhấn "✅ Mark as Reviewed" sau khi kiểm tra xong ảnh

### 💾 **Lưu kết quả**
- Nhấn "💾 Save All Changes" để lưu tất cả thay đổi

## 📝 **Các ký hiệu trạng thái**:
- **✅ Reviewed**: Ảnh đã được duyệt
- **🔄 Modified (X changes)**: Ảnh đã được chỉnh sửa X lần
- **❌ Not reviewed**: Ảnh chưa được duyệt
- **Xanh lá (SIG-X)**: Vùng chữ ký
- **Đỏ (NO-X)**: Vùng không phải chữ ký

## 💡 **Mẹo sử dụng**:
- Index bắt đầu từ 0 (SIG-0, SIG-1, NO-0, NO-1...)
- Giao diện tự động cập nhật sau mỗi thay đổi
- Lịch sử thay đổi được lưu để theo dõi
- Có thể chỉnh sửa nhiều lần trước khi lưu